In [ ]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

# Main TEST

In [242]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)
print(tensor_stimuli)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



[<granch_utils.init_model_tensor.granch_stimuli object at 0x142fb3880>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bfee0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bf430>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438be980>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bded0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bf3a0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bc310>, <granch_utils.init_model_tensor.granch_stimuli object at 0x1438bd4b0>]


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2550705207581827' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.011213854647874378' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.05418021111757588' has dtype incompatible with int64, please explicitly cast to a compatible 

AttributeError: 'DataFrame' object has no attribute 'behavior'

# Specific Grid Test

In [357]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)
from torch.distributions import Normal, uniform


# test stimuli
test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

# test grid  
grid_mu_distribution = uniform.Uniform(.2, .8)
grid_sigma_distribution = uniform.Uniform(.2, .8)
grid_y_distribution = uniform.Uniform(.2, .8)
grid_epsilon_distribution = uniform.Uniform(.2, .8)

grid_mu = grid_mu_distribution.sample([2, ])
grid_sigma = grid_sigma_distribution.sample([2, ])
grid_y = grid_y_distribution.sample([2, ])
grid_epsilon = grid_epsilon_distribution.sample([2, ])

#grid_mu = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_sigma = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_y = torch.linspace(start = .2, end  = .8, steps = 5)
#grid_epsilon = torch.linspace(start = .2, end  = .8, steps = 5)

# test prior
PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 0.00001, "max_observation": 500
}


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

params = init_params_tensor.granch_params(
                grid_mu =  grid_mu,
                grid_sigma = grid_sigma,
                grid_y = grid_y, 
                grid_epsilon = grid_epsilon,
                hypothetical_obs_grid_n = 5, 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29978886246681213' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999916672706604' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29996559023857117' has dtype incompatible with int64, please explicitly cast to a compatible d

In [339]:
res["kl1"][1]

1.4032541546255678e-07

In [ ]:
res

# Visualizing random grid KL (500)

In [368]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


def run_random_grid_and_visualize_kl():


    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    BATCH_INFO = {
        "jitter_n": 1, 
        "total_batch_n": 1, 
        "jitter_mode": "sampling"
    }

    GRID_INFO = {
       "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
       "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
       "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
       "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
       "hypothetical_obs_grid_n": 10
    }


#    GRID_INFO = {
#        "grid_mu_start": .2, "grid_mu_end": .8, "grid_mu_step": 5, 
#        "grid_sigma_start": .2, "grid_sigma_end": .8, "grid_sigma_step": 5, 
#        "grid_y_start":.2, "grid_y_end": 8, "grid_y_step": 5, 
#        "grid_epsilon_start": .2, "grid_epsilon_end": .8, "grid_epsilon_step": 5, 
#        "hypothetical_obs_grid_n": 10
#    }



    BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

    PRIOR_INFO = {
        "mu_prior": 1,  
        "V_prior": 1, 
        "alpha_prior": 3, 
        "beta_prior": 1, 
        "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
        "hypothetical_obs_grid_n": 2, 
        "world_EIGs": 0.0003, "max_observation": 500
    }


    test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBB")
    test_stimuli.add_toy_example(0.3, 0.7)
    



    tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

    params = init_params_tensor.granch_params(
                    grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                    grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                    grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                    grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                    hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                    mu_prior = PRIOR_INFO["mu_prior"],
                    V_prior = PRIOR_INFO["V_prior"], 
                    alpha_prior = PRIOR_INFO["alpha_prior"], 
                    beta_prior = PRIOR_INFO["beta_prior"],
                    epsilon  = PRIOR_INFO["epsilon"], 
                    mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                    sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                    world_EIGs = PRIOR_INFO["world_EIGs"],
                    max_observation = PRIOR_INFO["max_observation"], 
                    forced_exposure_max= np.nan, 
                    linking_hypothesis = "surprisal")
            
                # add the various different cached bits
    params.add_meshed_grid()
    params.add_lp_mu_sigma()
    params.add_y_given_mu_sigma()
    params.add_lp_epsilon()
    params.add_priors()

    if params.linking_hypothesis == "EIG": 
        res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
    else: 
        res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

    return res


#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



# try it out 5 eims
res_list = []
kl_df_list = []
for i in range(2000): 
    res, kl_df = run_random_grid_and_visualize_kl()
    res["id"] = i
    kl_df["id"] = i
    res_list.append(res)
    kl_df_list.append(kl_df)

main_res = pd.concat(res_list)
kl_df_res = pd.concat(kl_df_list)

main_res.to_csv("sim_res.csv")
kl_df_res.to_csv("sim_res_kl.csv")
#main_res.dropna()
#print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999837100505829' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3001415729522705' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29989054799079895' has dtype incompatible with int64, please explicitly cast to a compatible dt

1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
3
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
7
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
9
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
3
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
7
tensor

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29991886019706726' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2998470067977905' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29992008209228516' has dtype incompatible with int64, please explicitly cast to a compatible d

11
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
3
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
7
tensor(-7.4405e-17, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
9
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
11
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float6

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2997100055217743' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999584674835205' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2998594045639038' has dtype incompatible with int64, please explicitly cast to a compatible dty

...

11
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
3
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
7
tensor(-4.4039e-13, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
9
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
11
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
7
tensor(-9.6817e-12, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=tor

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29987189173698425' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000539541244507' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000030517578125' has dtype incompatible with int64, please explicitly cast to a compatible dt

5
tensor(-1.4293e-10, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
9
tensor(-5.5838e-11, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
8
tensor(-8.0405e-12, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
2
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
4
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
6
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
8
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
10
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], d

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29996421933174133' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29998987913131714' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29982006549835205' has dtype incompatible with int64, please explicitly cast to a compatible 

7
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
9
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
11
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
1
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
3
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(0., dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
0
tensor(-0.0284, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.1022, 1.1022, 1.1022])
3
tensor(-9.7227e-12, dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)
5
tensor(-3.34

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

df = main_res

# Get unique categories
id = df['id'].unique()
# Create a subplot grid
fig = sp.make_subplots(rows=len(id), cols=1, subplot_titles=["1", "2", "3", "4", "5"])

# Populate each subplot
for i, idx in enumerate(id):
    subset = df[df['id'] == idx]
    print(subset)
    single_fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)
    single_fig.update_layout(
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
    )


    for trace in single_fig.data:
        fig.add_trace(trace, row=i + 1, col=1)

# Update subplot layout (optional)
fig.update_layout(
    title='Faceted Line Plots',
    showlegend=True,
    height=600,
    width=400,
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px



df = res.iloc[:50]

# Create a line plot using Plotly Express
fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)

# Customize the layout (optional)
fig.update_layout(
    title='Line Plot with Plotly',
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
)

# Show the plot
fig.show()


# Toy Example Test 

In [179]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = 3,
                #world_EIGs = PRIOR_INFO["world_EIGs"],
                #max_observation = PRIOR_INFO["max_observation"], 
                max_observation = 500,
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

pd.set_option('display.max_rows', None)

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
#res.behavior

res.behavior

NameError: name 'tensor_stimuli' is not defined

In [ ]:
res.behavior

In [303]:
import torch.nn.functional as F

new_post = torch.tensor([[[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1620e-02, 9.9999e-321]]],


        [[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1619e-02, 9.9999e-321]]],


        [[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1620e-02, 9.9999e-321]]]], dtype=torch.float64)

paded_prev_post = torch.tensor([[[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]],


        [[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]],


        [[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]]])

#print(new_post/paded_prev_post)
#print(torch.log(new_post/paded_prev_post))

KL1 = torch.sum(torch.mul(new_post,
                         torch.log(new_post/paded_prev_post)), dim = (1, 2, 3))

KL2 = F.kl_div(torch.log(paded_prev_post),new_post, reduction = "none").sum(dim = (1, 2, 3))

print(KL1)
print(KL2)

print(new_post.sum())
print(paded_prev_post.sum(dim = (1, 2, 3)))
print(new_post < paded_prev_post)

print(torch.log(new_post/paded_prev_post))

tensor([-0.8116, -0.8116, -0.8116], dtype=torch.float64)
tensor([-0.8116, -0.8116, -0.8116], dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([2.2707, 2.2707, 2.2707])
tensor([[[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]],


        [[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]],


        [[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]]])
tensor([[[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]],


        [[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]],


        [[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]]], dtype=torch.float64)


# Test KL overflow

In [241]:

A = torch.tensor([9.9999e-321,  2.4348e-01])
B = torch.tensor([1.0000e-31, 3.6788e-01])
        
A_cast_first = torch.tensor([9.9999e-321,  2.4348e-01], dtype=torch.float64)
B_cast_first = torch.tensor([1.0000e-31, 3.6788e-01], dtype=torch.float64)  

A_cast_later = A.to(torch.float64)
B_cast_later = B.to(torch.float64)


mask = A_cast_later < 1e-10
A_cast_later[mask] = 1e-10 
mask = B_cast_later < 1e-10
B_cast_later[mask] = 1e-10


KL1 = torch.sum(torch.mul(A,torch.log(A/B)))
KL2 = torch.sum(torch.mul(A_cast_later,torch.log(A_cast_later/B_cast_later)))
KL3 =  torch.sum(torch.mul(A_cast_first,torch.log(A_cast_first/B_cast_first)))

print(KL1)
print(KL2)
print(KL3)

tensor(nan)
tensor(-0.1005, dtype=torch.float64)
tensor(-0.1005, dtype=torch.float64)


In [168]:
A = torch.tensor([9.9999e-321])
B = A.to(torch.float64)
print(A)
print(B)
print(B.dtype)

tensor([0.])
tensor([0.], dtype=torch.float64)
torch.float64


In [124]:
pred = torch.tensor([[0.2, 0.8]])
target = torch.tensor([[0.1, 0.9]])

kl2_a = F.kl_div(pred, target, reduction='sum', log_target=False)

kl2_b = F.kl_div(target, pred, reduction='sum', log_target=True)

KL1 = torch.sum(torch.mul(new_post, torch.log(new_post/paded_prev_post)), dim = 0)

print(kl2_a)
print(kl2_b)

print(torch.sum(torch.exp(target) * (target - pred)))


tensor(-1.0651)
tensor(-0.1004)
tensor(0.1354)
